In [1]:
#!pip3 install openmeteo-requests retry-requests geopy

In [2]:
import json
with open('config.json') as f:
    secrets = json.load(f)

In [3]:
import boto3
from botocore.exceptions import ClientError
import openmeteo_requests
import json

import pandas as pd
from retry_requests import retry
from requests import Session

client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2",aws_access_key_id=secrets['aws_access_key_id'], aws_secret_access_key=secrets['aws_secret_access_key']) 
#model_id = "meta.llama3-1-70b-instruct-v1:0"
model_id = "meta.llama3-1-405b-instruct-v1:0"
#model_id = "anthropic.claude-3-opus-20240229-v1:0"


In [4]:
radio_tools = {
    "tools": [
        {
            "toolSpec": {
                "name": "top_song",
                "description": "Get the most popular song played on a radio station.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "sign": {
                                "type": "string",
                                "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ and WKRP."
                            }
                        },
                        "required": [
                            "sign"
                        ]
                    }
                }
            }
        }
    ]
}

conversation = [
    {
        "role": "user",
        "content": [
            {
                "text": "What is the most popular song on WZPZ?"
            }
        ]
    }
]


# Send the message to the model, using a basic inference configuration.
response = client.converse(
    toolConfig = radio_tools,
    modelId=model_id,
    messages=conversation,
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
)


In [5]:
response

{'ResponseMetadata': {'RequestId': 'f2665929-c43f-42ba-84eb-f0a544472338',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 13 Aug 2024 20:45:56 GMT',
   'content-type': 'application/json',
   'content-length': '273',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f2665929-c43f-42ba-84eb-f0a544472338'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'toolUse': {'toolUseId': 'tooluse_s7PwcTzdTGuLEq_LVKUmCw',
      'name': 'top_song',
      'input': {'sign': 'WZPZ'}}}]}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 103, 'outputTokens': 28, 'totalTokens': 131},
 'metrics': {'latencyMs': 2711}}

In [6]:
# explicit function
def fun(a: int, b: int):
    return a**b

In [7]:
import inspect

def function_inspection(func) -> dict:
    field_types = {
        str: "string",
        int: "integer",
        float: "float",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Error retrieving function signature for {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = field_types.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type {param.annotation} for {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]
    if "kwargs" in required: required.remove("kwargs")

    return {
        "toolSpec": {
            "name": func.__name__,
            "description": func.__doc__ or "",
            "inputSchema": {
                "json": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
            },
        },
    }


In [8]:
function_inspection(fun)

{'toolSpec': {'name': 'fun',
  'description': '',
  'inputSchema': {'json': {'type': 'object',
    'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
    'required': ['a', 'b']}}}}

In [9]:
import openmeteo_requests

import pandas as pd
from retry_requests import retry
from requests import Session

# Setup the Open-Meteo API client with cache and retry on error
retry_session = retry(Session(), retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m
0   2024-08-13 00:00:00+00:00       17.645500
1   2024-08-13 01:00:00+00:00       16.945499
2   2024-08-13 02:00:00+00:00       16.445499
3   2024-08-13 03:00:00+00:00       16.145500
4   2024-08-13 04:00:00+00:00       15.595500
..                        ...             ...
163 2024-08-19 19:00:00+00:00       19.769501
164 2024-08-19 20:00:00+00:00       18.819500
165 2024-08-19 21:00:00+00:00       17.969501
166 2024-08-19 22:00:00+00:00       17.219501
167 2024-08-19 23:00:00+00:00       16.519501

[168 rows x 2 columns]


In [10]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"current": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m}")

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current time 1723581900
Current temperature_2m 22.200000762939453


In [11]:
def current_weather(latitude:float, longitude:float, place:str="") -> str:
    """Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in"""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    current = response.Current()
    current_temperature_2m = current.Variables(0).Value()

    return "The current temperature at %s is %f degrees." %(place, current_temperature_2m)

In [12]:
current_weather(47.4864, 122.1943)

'The current temperature at  is 14.850000 degrees.'

In [13]:
function_inspection(current_weather)

{'toolSpec': {'name': 'current_weather',
  'description': "Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in",
  'inputSchema': {'json': {'type': 'object',
    'properties': {'latitude': {'type': 'float'},
     'longitude': {'type': 'float'},
     'place': {'type': 'string'}},
    'required': ['latitude', 'longitude']}}}}

In [14]:
policy = """You are a weather assistant agent - users ask for a variety of information including the temperature, humidity, and preciptation that is weather related.  
You have access to current, past, and future weather using Functions, 
do not answer from memory for weather related information, instead collect the required data and call the Function appropriately"""

In [15]:
from pydantic import BaseModel, Field
from typing import List, Callable, Dict
class Agent(BaseModel):
    name: str = 'Agent'
    llm_model_id: str = "meta.llama3-1-405b-instruct-v1:0"
    policy: str = "You are an AI assistant."
    tools: Dict = {}
    tools_json: Dict = {}

    def build_tool_json(self):
        tools_json = {"tools": []}
        for k,v in self.tools.items():
            tools_json['tools'].append(function_inspection(v))
        self.tools_json = tools_json


In [16]:
weather_agent = Agent(name="Weather Agent", policy=policy,tools={"current_weather": current_weather})

In [17]:
weather_agent.build_tool_json()

In [18]:
weather_agent.tools_json

{'tools': [{'toolSpec': {'name': 'current_weather',
    'description': "Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in",
    'inputSchema': {'json': {'type': 'object',
      'properties': {'latitude': {'type': 'float'},
       'longitude': {'type': 'float'},
       'place': {'type': 'string'}},
      'required': ['latitude', 'longitude']}}}}]}

In [19]:
# Start a conversation with the user message.
user_message = "What's the temperature for lat 47.4864 and long 122.1943?"
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]


# Send the message to the model, using a basic inference configuration.
response = client.converse(
    system=[{"text": weather_agent.policy}],
    modelId=weather_agent.llm_model_id,
    messages=conversation,
    toolConfig = weather_agent.tools_json,
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},)


In [20]:
response

{'ResponseMetadata': {'RequestId': '5701db32-c1ad-4b6a-ba34-f6374dfab7e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 13 Aug 2024 20:46:00 GMT',
   'content-type': 'application/json',
   'content-length': '300',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5701db32-c1ad-4b6a-ba34-f6374dfab7e3'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'toolUse': {'toolUseId': 'tooluse_peUVF1ehQOG-fFXGK6GrQg',
      'name': 'current_weather',
      'input': {'latitude': '47', 'longitude': '122'}}}]}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 172, 'outputTokens': 31, 'totalTokens': 203},
 'metrics': {'latencyMs': 2883}}

In [21]:
weather_agent.tools["current_weather"](**{'latitude': '47', 'longitude': '122'})

'The current temperature at  is 16.950001 degrees.'

In [22]:
weather_agent.tools

{'current_weather': <function __main__.current_weather(latitude: float, longitude: float, place: str = '') -> str>}

In [23]:
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
    ]

def run_inference(conversation,agent):
    response = client.converse(
        system=[{"text": agent.policy}],
        modelId=agent.llm_model_id,
        messages=conversation,
        toolConfig = agent.tools_json,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},)
    
    print(json.dumps(response, indent=2))

    while "toolUse" in response['output']['message']['content'][0].keys():
        tool_name = response['output']['message']['content'][0]['toolUse']['name']
        tool_inputs = response['output']['message']['content'][0]['toolUse']['input']

        tool_result = weather_agent.tools[tool_name](**tool_inputs)

        print("called tool with result " + str(tool_result))

        conversation.append({"role":"user","content":tool_result})

        return conversation
    
    else:
        response_text = response["output"]["message"]["content"][0]["text"]
        print(response_text)

        conversation.append({"role": "assistant","content":response_text})

        return conversation

In [24]:
class Inference(BaseModel):
    conversation: List
    agent: Agent
    system_prompt:str = ""

    def set_prompt(self):
        self.system_prompt = self.agent.policy


    def run_inference(self):
        response = client.converse(
            system=[{"text": self.system_prompt}],
            modelId=self.agent.llm_model_id,
            messages=self.conversation,
            toolConfig = self.agent.tools_json,
            inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},)

        #print(self.system_prompt, self.conversation)
        print(json.dumps(response, indent=2))

        while "toolUse" in response['output']['message']['content'][0].keys():
            tool_name = response['output']['message']['content'][0]['toolUse']['name']
            tool_inputs = response['output']['message']['content'][0]['toolUse']['input']

            tool_result = weather_agent.tools[tool_name](**tool_inputs)

            print("called tool with result " + str(tool_result) + " Is this enough information to answer the question?")

            #self.system_prompt = self.system_prompt+" "+tool_result
            self.conversation.append({"role": "assistant",'content': [{'text':"Called tool %s with result %s" %(tool_name, tool_result)}]})
            self.conversation.append({"role": "user",'content': [{'text':"Is this enough information to answer the question?"}]})

            #print(self.system_prompt)

            self.run_inference()
        
        response_text = response["output"]["message"]["content"][0]["text"]
        print(response_text)

        self.conversation.append({"role": "assistant",'content': [{'text':response_text}]})

        print(self.conversation)
        return response_text

In [25]:
class Inference(BaseModel):
    conversation: List
    agent: Agent
    system_prompt:str = ""

    def set_prompt(self):
        self.system_prompt = self.agent.policy

    def run_inference(self):
        response = client.converse(
            system=[{"text": self.system_prompt}],
            modelId=self.agent.llm_model_id,
            messages=self.conversation,
            toolConfig = self.agent.tools_json,
            inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},)

        print(json.dumps(response, indent=2))
        
        if "toolUse" in response['output']['message']['content'][0].keys():
            tool_name = response['output']['message']['content'][0]['toolUse']['name']
            tool_inputs = response['output']['message']['content'][0]['toolUse']['input']

            tool_result = weather_agent.tools[tool_name](**tool_inputs)

            print("called tool with result " + str(tool_result) + " Is this enough information to answer the question?")

            #self.system_prompt = self.system_prompt+" "+tool_result
            self.conversation.append({"role": "assistant",'content': [{'text':"Called tool %s with result %s" %(tool_name, tool_result)}]})
            self.conversation.append({"role": "user",'content': [{'text':"Is this enough information to answer the question?"}]})

            return("tool_call")
        
        else:
            response_text = response["output"]["message"]["content"][0]["text"]
            print(response_text)

            self.conversation.append({"role": "assistant",'content': [{'text':response_text}]})

            print(self.conversation)

        return response_text

    def run(self):
        response = self.run_inference()
        while response == "tool_call":
            response = self.run_inference()
        print(response)


In [26]:
policy = """You are a weather assistant agent - users ask for a variety of information including the temperature, humidity, and preciptation that is weather related.  
Use tools where necessary to gather more information if it is not provided by the user or in this prompt."""

weather_agent = Agent(name="Weather Agent", policy=policy,tools={"current_weather": current_weather, "geolocation":geolocation})
weather_agent.build_tool_json()

NameError: name 'geolocation' is not defined

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [{"text": "What's the current temperature in Atlanta Georgia?"}],
    }
    ]

weather_conversation = Inference(conversation=conversation, agent=weather_agent)
weather_conversation.set_prompt()
weather_conversation.run()

{
  "ResponseMetadata": {
    "RequestId": "4d2d38bd-3027-4729-b5c6-3d5f43858d1c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 13 Aug 2024 03:47:47 GMT",
      "content-type": "application/json",
      "content-length": "292",
      "connection": "keep-alive",
      "x-amzn-requestid": "4d2d38bd-3027-4729-b5c6-3d5f43858d1c"
    },
    "RetryAttempts": 1
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "toolUse": {
            "toolUseId": "tooluse_Dmx4eutbRwaQKqu6VkWDKw",
            "name": "geolocation",
            "input": {
              "location": "Atlanta, Georgia"
            }
          }
        }
      ]
    }
  },
  "stopReason": "tool_use",
  "usage": {
    "inputTokens": 218,
    "outputTokens": 27,
    "totalTokens": 245
  },
  "metrics": {
    "latencyMs": 2591
  }
}
called tool with result The latitude and longitude for Atlanta, Georgia is 33.748992, -84.390264 Is this enough information to a

In [ ]:
from geopy.geocoders import Nominatim
def geolocation(location: str, **kwargs) -> str:
    """Input user location with as much specificity as is known, if a city and a state are provided, submit in the form of City, State"""
    geolocator = Nominatim(user_agent="location_agent")
    loc = geolocator.geocode(location)
    return "The latitude and longitude for %s is %f, %f" % (location, loc.latitude, loc.longitude)

In [ ]:
policy = """You are a weather assistant agent - users ask for a variety of information including the temperature, humidity, and preciptation that is weather related.  
Use tools where necessary to gather more information if it is not provided by the user or in this prompt"""

weather_agent = Agent(name="Weather Agent", policy=policy,tools={"current_weather": current_weather, "geolocation":geolocation})
weather_agent.build_tool_json()

In [ ]:
print(json.dumps(weather_agent.tools_json,indent=2))

{
  "tools": [
    {
      "toolSpec": {
        "name": "current_weather",
        "description": "Gets the current temperature for a location given a specific location's latitude and longitude, pass the full floating point number values in",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "latitude": {
                "type": "float"
              },
              "longitude": {
                "type": "float"
              }
            },
            "required": [
              "latitude",
              "longitude"
            ]
          }
        }
      }
    },
    {
      "toolSpec": {
        "name": "geolocation",
        "description": "Input user location with as much specificity as is known, if a city and a state are provided, submit in the form of City, State",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "location": {
             

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [{"text": "What's the weather in Atlanta Georgia?"}],
    }
    ]

run_inference(conversation,weather_agent)

{
  "ResponseMetadata": {
    "RequestId": "ed6e7d12-b803-4cf7-9967-0cd411111c00",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 13 Aug 2024 01:33:34 GMT",
      "content-type": "application/json",
      "content-length": "292",
      "connection": "keep-alive",
      "x-amzn-requestid": "ed6e7d12-b803-4cf7-9967-0cd411111c00"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "toolUse": {
            "toolUseId": "tooluse_Nfr5eiHyQP6U12h1hgnbdA",
            "name": "geolocation",
            "input": {
              "location": "Atlanta, Georgia"
            }
          }
        }
      ]
    }
  },
  "stopReason": "tool_use",
  "usage": {
    "inputTokens": 230,
    "outputTokens": 27,
    "totalTokens": 257
  },
  "metrics": {
    "latencyMs": 2690
  }
}
called tool with result The latitude and longitude for Atlanta, Georgia is 33.748992, -84.390264


[{'role': 'user',
  'content': [{'text': "What's the weather in Atlanta Georgia?"}]},
 {'role': 'user',
  'content': 'The latitude and longitude for Atlanta, Georgia is 33.748992, -84.390264'}]

In [ ]:
# Start a conversation with the user message.
user_message = "What's the lat long for Renton, WA."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)



The latitude and longitude for Renton, WA is:

Latitude: 47.4829° N
Longitude: 122.2171° W

Note: These coordinates are for the city center of Renton, WA. If you need more specific coordinates for a particular location within Renton, please let me know and I can try to provide those for you.


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")
city ="Fairwood, WA"
loc = geolocator.geocode(city)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 47.4468672 
longtitude is:- -122.1525349


In [ ]:
loc.latitude, loc.longitude

(47.4468672, -122.1525349)

In [ ]:
client.converse??

Signature: client.converse(*args, **kwargs)
Docstring:
Sends messages to the specified Amazon Bedrock model. ``Converse`` provides a consistent interface that works with all models that support messages. This allows you to write code once and use it with different models. If a model has unique inference parameters, you can also pass those unique parameters to the model.

 

Amazon Bedrock doesn't store any text, images, or documents that you provide as content. The data is only used to generate the response.

 

For information about the Converse API, see *Use the Converse API* in the *Amazon Bedrock User Guide*. To use a guardrail, see *Use a guardrail with the Converse API* in the *Amazon Bedrock User Guide*. To use a tool with a model, see *Tool use (Function calling)* in the *Amazon Bedrock User Guide*

 

For example code, see *Converse API examples* in the *Amazon Bedrock User Guide*.

 

This operation requires permission for the ``bedrock:InvokeModel`` action.



See also: `AWS